### Introduction to Machine Learning in Spark using MLlib

In this notebook we demonstrate some of the basics of machine learning using Spark's MLlib library. The examples are taken from Spark's documentation. Full details can be found [here](https://spark.apache.org/docs/latest/ml-pipeline.html) along with the equivalent code in the scala programming language. 

Topics
- MLlib
- Vectors
- Transformers
- Estimators
- Models
- Pipelines
- Pipeline models
- Parameters

### Example from MLlib documentation (no pipeline)

This example uses Logistic Regression for classification. It highlights some of the most important classes in MLlib
 - Transformers
 - Estimators
 - Parameters

**Note:** The first paragraph shows the format of the DataFrame columns required for machine learning with MLlib
- label (or dependent variable) in 'Double'
- features (independent variables) in a `Vector`
- The default column names for these are `label` and `features` respectively

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Prepare training data from a list of (label, features) tuples
# Note that MLlib requires the label (or dependent variable) to be of type double

# The coulumn names "label" and "features" are the defaults in MLlib 
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

In [0]:
training.printSchema()

root
-- label: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
training.show()

+-----+--------------+
label| features|
+-----+--------------+
 1.0| [0.0,1.1,0.1]|
 0.0|[2.0,1.0,-1.0]|
 0.0| [2.0,1.3,1.0]|
 1.0|[0.0,1.2,-0.5]|
+-----+--------------+

Create a Logistic Regression instance. This instance is an **Estimator**

In [0]:
# Create a LogisticRegression instance
lr = LogisticRegression(maxIter=10, regParam=0.01)

Print out the parameters, documentation, and any default values

In [0]:
print("Logistic Regression parameters:\n" + lr.explainParams() + "\n")

Logistic Regression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal with 1 for binomial regression, or the number oflasses for multinomial regression. (undefined)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0, current: 0.01)
standardization: whether to standardize the training features before fitting the model. (default: True)
threshold: Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p]. (default: 0.5)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold. (undefined)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
upperBoundsOnCoefficients: The upper bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
upperBoundsOnIntercepts: The upper bounds on intercepts if fitting under bound constrained optimization. The bound vector size must be equal with 1 for binomial regression, or the number of classes for multinomial regression. (undefined)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

Learn a LogisticRegression model

In [0]:
# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Note that even though this is a toy problem fitting the model initiates serveral Spark jobs (fitting a model requires many iterations)

Since `model1` is a Model (i.e. a transformer produced by an Estimator), we can view the parameters it used during `fit()`.

Note that the values for `maxIter` and `regParam` are as specified above. The other parameters are at their default values

In [0]:
# Print the parameter (name: value) pairs, where names are unique IDs for
# this LogisticRegression instance

print("Model 1 was fit using parameters: ")
params = lr.extractParamMap()
for item in params:
    print(item.name, params[item])
    # print(item.name, item.doc, params[item]) # item.doc provides a short description of the parameter

Model 1 was fit using parameters: 
maxBlockSizeInMB 0.0
threshold 0.5
aggregationDepth 2
standardization True
fitIntercept True
elasticNetParam 0.0
predictionCol prediction
featuresCol features
labelCol label
rawPredictionCol rawPrediction
probabilityCol probability
maxIter 10
regParam 0.01
tol 1e-06
family auto

We may alternatively specify parameters using a Python dictionary as a paramMap

In [0]:
# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)
    
# print (paramMapCombined)
params = paramMapCombined
for item in params:
    print(item.name, params[item])


maxIter 30
regParam 0.1
threshold 0.55
probabilityCol myProbability

Now learn a new model using the paramMapCombined parameters.

`paramMapCombined` overrides all parameters set earlier via `lr.set*` methods

In [0]:
model2 = lr.fit(training, paramMapCombined)
print ("model2 is of type: "+ str(type(model2)))

model2 is of type: <class 'pyspark.ml.classification.LogisticRegressionModel'>

In [0]:
print("Model 2 was fit using parameters: ")
params = lr.extractParamMap(extra = paramMapCombined)
for item in params:
    print(item.name, params[item])

Model 2 was fit using parameters: 
maxBlockSizeInMB 0.0
threshold 0.55
aggregationDepth 2
standardization True
fitIntercept True
elasticNetParam 0.0
predictionCol prediction
featuresCol features
labelCol label
rawPredictionCol rawPrediction
probabilityCol myProbability
maxIter 30
regParam 0.1
tol 1e-06
family auto

In [0]:
# Examine the model coefficents
print(model2.coefficientMatrix)
print(model2.coefficients)

DenseMatrix([[-1.43136588, 0.43208871, -0.14920419]])

[-1.4313658815706811,0.43208871014875283,-0.14920419477974745]

Make predictions on test data using the `Transformer.transform()` method. LogisticRegression.transform will only use the 'features' column.
Note that `model2.transform()` outputs a "myProbability" column instead of the usual 'probability' column since we renamed the lr.probabilityCol parameter previously.

In [0]:
# Prepare test data
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

prediction = model2.transform(test)

result = prediction.select("features", "label", "myProbability", "prediction") \
    .collect()

for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.myProbability, row.prediction))

features=[-1.0,1.5,1.3], label=1.0 -> prob=[0.057073041710340174,0.9429269582896598], prediction=1.0
features=[3.0,2.0,-0.1], label=0.0 -> prob=[0.9238522311704104,0.07614776882958962], prediction=0.0
features=[0.0,2.2,-1.5], label=1.0 -> prob=[0.10972776114779444,0.8902722388522055], prediction=1.0

In [0]:
prediction.show()

+-----+--------------+--------------------+--------------------+----------+
label| features| rawPrediction| myProbability|prediction|
+-----+--------------+--------------------+--------------------+----------+
 1.0|[-1.0,1.5,1.3]|[-2.8046569418746...|[0.05707304171034...| 1.0|
 0.0|[3.0,2.0,-0.1]|[2.49587635664205...|[0.92385223117041...| 0.0|
 1.0|[0.0,2.2,-1.5]|[-2.0935249027913...|[0.10972776114779...| 1.0|
+-----+--------------+--------------------+--------------------+----------+

In [0]:
prediction.head(3)

Out[39]: [Row(label=1.0, features=DenseVector([-1.0, 1.5, 1.3]), rawPrediction=DenseVector([-2.8047, 2.8047]), myProbability=DenseVector([0.0571, 0.9429]), prediction=1.0),
 Row(label=0.0, features=DenseVector([3.0, 2.0, -0.1]), rawPrediction=DenseVector([2.4959, -2.4959]), myProbability=DenseVector([0.9239, 0.0761]), prediction=0.0),
 Row(label=1.0, features=DenseVector([0.0, 2.2, -1.5]), rawPrediction=DenseVector([-2.0935, 2.0935]), myProbability=DenseVector([0.1097, 0.8903]), prediction=1.0)]

### Example from MLlib documentation (with pipeline)

MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single `pipeline`, or workflow. MLlib's Pipelines API was inspired by the scikit-learn project.

This example illustrates a machine learning pipeline to chain multiple Transformers and Estimators together to specify a simple ML workflow. It connects:
- DataFrames: This ML API uses DataFrames from Spark SQL as an ML dataset, which can hold a variety of data types. e.g., a DataFrame could have different columns storing text, feature vectors, true labels, and predictions.
- Transformers: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. e.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.
- Estimator: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. e.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.
- Parameter: All Transformers and Estimators share a common API for specifying parameters.

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [0]:
# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(training)

print(type(model))

<class 'pyspark.ml.pipeline.PipelineModel'>

In [0]:
# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [0]:
# Make predictions on test documents and examine resulting dataframe
prediction2 = model.transform(test)

# prediction2.show()
prediction2.head()

Out[43]: Row(id=4, text='spark i j k', words=['spark', 'i', 'j', 'k'], features=SparseVector(262144, {19036: 1.0, 68693: 1.0, 173558: 1.0, 213660: 1.0}), rawPrediction=DenseVector([-1.6609, 1.6609]), probability=DenseVector([0.1596, 0.8404]), prediction=1.0)

In [0]:
# Print columns of interest
selected = prediction2.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(4, spark i j k) --> prob=[0.15964077387874095,0.840359226121259], prediction=1.000000
(5, l m n) --> prob=[0.8378325685476612,0.1621674314523388], prediction=0.000000
(6, spark hadoop spark) --> prob=[0.06926633132976255,0.9307336686702374], prediction=1.000000
(7, apache hadoop) --> prob=[0.9821575333444208,0.017842466655579203], prediction=0.000000

###Hands On

![Hands-on](https://cis442f-open-data.s3.amazonaws.com/pictures/hands.png "Hands-on")


#### Exercises

Work through this code line by line.
1. What are DataFames?
2. What are Transformers?
3. What are Estimators?
4. What are Pipelines?
5. How do you set model parameters?